In [ ]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras.layers import Average, Dense, Dropout , Concatenate, Input
from keras.optimizers import SGD,Adam,RMSprop
from keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras import Model
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_squared_error

# data processing

In [ ]:
df = pd.read_csv('../input/housesales/HouseSales.csv',nrows=5000)
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.corr()

In [ ]:
df.head()

In [ ]:
def edit(x):
    x = str(x).split('T')[0]
    return x

df['date'] = df['date'].apply(edit)

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
df.date = pd.to_numeric(df.date)
df.head()

In [ ]:
df[['date','price']].corr()

In [ ]:
df.drop(columns=['id','date','zipcode','lat','long'],inplace = True)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df[df.yr_renovated == 0].shape

In [ ]:
df.drop(columns=['yr_renovated'],inplace = True)
df.head()

In [ ]:
df.bedrooms.value_counts()

In [ ]:
df[df.bedrooms == 0]

In [ ]:
df.bedrooms.value_counts()

In [ ]:
df.bedrooms.mean() + 3 * df.bedrooms.std()

In [ ]:
df.drop(df[df.bedrooms == 0].index,inplace = True)
df.drop(df[df.bedrooms >= 7].index,inplace = True)

In [ ]:
df.bedrooms.value_counts()

In [ ]:
df.bathrooms.value_counts()

In [ ]:
df.bathrooms.value_counts().tail(9).index

In [ ]:
df.drop(df[df['bathrooms'].isin(df.bathrooms.value_counts().tail(9).index)].index,inplace = True)

In [ ]:
df.bathrooms.value_counts()

In [ ]:
df['floors'].value_counts()

In [ ]:
df.grade.value_counts()

In [ ]:
df.drop(df[df['grade'].isin(df.grade.value_counts().tail(3).index)].index,inplace = True)

In [ ]:
df.waterfront.value_counts()

In [ ]:
df[df.waterfront == 1]

In [ ]:
df.drop(columns = ['waterfront'],inplace = True)

In [ ]:
df.view.value_counts()

In [ ]:
df.condition.value_counts()

In [ ]:
df.drop(df[df.condition == 1].index,inplace = True)

In [ ]:
df.yr_built.min()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
for cols in ['price', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement',
       'yr_built', 'sqft_living15', 'sqft_lot15']:
    df[cols] = (df[cols] - df[cols].min()) / (df[cols].max() - df[cols].min())

In [ ]:
for cols in ['bedrooms','bathrooms','floors','view','condition','grade']:
    df[cols] = df[cols].astype('category')
    
df.info()

In [ ]:
dummy_df = pd.get_dummies(df[['bedrooms','bathrooms','floors','view','condition','grade']])
dummy_df.head()

In [ ]:
df = pd.concat([df, dummy_df], axis=1)
df.drop(columns = ['bedrooms','bathrooms','floors','view','condition','grade'], inplace=True)
df.head()

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
features = df.drop(columns = ['price']).values
target = df.price.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features,target ,test_size = 0.2 ,random_state=42)

In [ ]:
print('train dataset shape : features - {} , labels {}'.format(X_train.shape,y_train.shape))
print('test dataset shape : features - {} , labels {}'.format(X_test.shape,y_test.shape))

In [ ]:
def make_ANN(optim = 'sgd',lr = 0.1):
    input1 = Input(50 , name = 'InputLayer')
    model1 = Dense(128, activation='relu', name='HiddenLayer1')(input1)
    model1 = Dense(32, activation='relu', name='HiddenLayer2')(model1)
    output1 = Dense(1, activation = 'linear',name = 'OutputLayer')(model1)

    ANN1 = Model(inputs= [input1], outputs=[output1])
    
    if optim == 'sgd':
        sgd = SGD(lr=lr, momentum=0.99)
        ANN1.compile(optimizer=sgd, loss='mse', metrics=['mse'])
        return ANN1
    
    elif optim == 'adam':
        adam = Adam(lr=lr)
        ANN1.compile(optimizer=adam, loss='mse', metrics=['mse'])
        return ANN1
    
    elif optim == 'rmsprop':
        rmsprop = RMSprop(lr = lr)
        ANN1.compile(optimizer=rmsprop, loss='mse', metrics=['mse'])
        return ANN1

In [ ]:
ANN_sgd = make_ANN(optim = 'sgd',lr = 0.1)
ANN_sgd.summary()

In [ ]:
plot_model(ANN_sgd, show_shapes=True)

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   mode='auto',
                   verbose=1,
                   patience=5)

lrr= ReduceLROnPlateau(monitor='val_loss',
                       factor=0.1, 
                       patience=3, 
                       min_lr=1e-11)

In [ ]:
counter = 0
def train(model,batch_size,counter):
    history = model.fit(x= X_train, y = y_train,
                              epochs= 200,
                              validation_split = 0.2,
                              batch_size = batch_size,  
                              verbose=0,
                              callbacks=[es , lrr])
    val_loss = history.history['val_loss']
    loss = history.history['loss']

    plt.figure(figsize=(15,10))
    plt.plot(val_loss)
    plt.plot(loss)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Val error','Train error'], loc='upper right')
    plt.savefig('train_error{}.png'.format(counter))
    plt.show()
    
    pred = model.predict(X_test)
    print('RMSE  :' , mean_squared_error(pred, y_test, squared=False))

In [ ]:
ANN = make_ANN(optim = 'sgd',lr = 0.1)
train(model = ANN,batch_size= 32,counter = counter )
counter  = counter + 1

In [ ]:
ANN = make_ANN(optim = 'sgd',lr = 0.01)
train(model = ANN,batch_size= 32,counter = counter )
counter  = counter + 1

In [ ]:
ANN = make_ANN(optim = 'sgd',lr = 0.001)
train(model = ANN,batch_size= 32,counter = counter )
counter  = counter + 1

In [ ]:
ANN = make_ANN(optim = 'sgd',lr = 0.001)
train(model = ANN,batch_size= 16,counter = counter )
counter  = counter + 1

In [ ]:
def make_ANN(optim = 'sgd',lr = 0.1):
    input1 = Input(50 , name = 'InputLayer')
    model1 = Dense(128, activation='relu', name='HiddenLayer1')(input1)
    output1 = Dense(1, activation = 'linear',name = 'OutputLayer')(model1)

    ANN1 = Model(inputs= [input1], outputs=[output1])
    
    if optim == 'sgd':
        sgd = SGD(lr=lr, momentum=0.99)
        ANN1.compile(optimizer=sgd, loss='mse', metrics=['mse'])
        return ANN1
    
    elif optim == 'adam':
        adam = Adam(lr=lr)
        ANN1.compile(optimizer=adam, loss='mse', metrics=['mse'])
        return ANN1
    
    elif optim == 'rmsprop':
        rmsprop = RMSprop(lr = lr)
        ANN1.compile(optimizer=rmsprop, loss='mse', metrics=['mse'])
        return ANN1

In [ ]:
ANN = make_ANN(optim = 'sgd',lr = 0.001)
train(model = ANN,batch_size= 32,counter = counter )
counter  = counter + 1

In [ ]:
ANN = make_ANN(optim = 'sgd',lr = 0.001)
train(model = ANN,batch_size= 16,counter = counter )
counter  = counter + 1